In [ ]:
pip install faker

In [5]:
import random
from faker import Faker
from faker.providers import phone_number
import re

### Задание

Пополнить список подозреваемых (начальный в файле suspects.txt) на основе следующего правила: **подозреваемым считается человек, который звонил другому подозреваемому и звонок длился более 2x минут.**

Решить задачу так, чтобы поиск подозреваемых проводился несколько раз (**формируя N-е круги подозреваемых**, т.е. в файле 0-й круг, те, кто звонил им - 1й круг, кто звонил подозреваемым 1-го круга - 2й круг и т.д.) **Для этого увеличить количество звонков до 10-20 тысяч и перегенерировать.**

**Увеличить количество звонков до 100 тысяч и усложнить правило для подозреваемых - требуется минимум 3 звонка длительностью более 2х минут**, чтобы добавить абонента в список подозреваемых.

 Результаты сохранить в файл (в suspects), разбить на "круги" не потеряв при этом возможность читать файл. (валидация входящей строки). И файл с табличкой "доказательств" куда должны попасть звонки, на основе которых вы сделали вывод, что тот или иной абонент должен попасть в список подозреваемых. Ну и нормализованные номера телефонов тоже в новый файлик

In [36]:
def generation_data(num_numbers = 100, num_calls = 15000):
  """
  Генерирование фальшивых звонков.
  num_numbers: количество уникальных сгенированных номеров телефонов
  num_calls: количество сгенированных звонков вида: caller, recipient, duration_s
  calls -> выходные сгенерированные данные звонков
  """

  fake = Faker()
  fake.add_provider(phone_number)
  pnumbers = [fake.unique.phone_number() for i in range(num_numbers)] # генерация уникальных номеров с заданным количеством
  # генерация звонков с заданным количеством
  calls = [ {"caller": random.choice(pnumbers), "recipient": random.choice(pnumbers), "duration_s": random.randint(5, 600)} for i in range(num_calls) ] 

  # замена входящего номера, если номера исходящих и входящих номеров совпадают
  for i in calls:
    while i['caller'] == i['recipient']:
      i['recipient'] = random.choice(pnumbers)

  # with open("calls.txt", "w+") as file:
  #  file.writelines([f'caller:{call.get("caller")}|recipient:{call.get("recipient")}|duration_s:{call.get("duration_s")}\n' for call in calls])


  # форматирование номеров:
  # учитывая локацию US умолчанию (https://github.com/joke2k/faker/blob/master/faker/providers/phone_number/en_US/__init__.py),
  # 1. отбрасывается добавочное значение после х
  # 2. удаляются все символы, кроме цифр
  # 3. отбираются последние 11 цифр, так как +1, 001 коды US
  
  suspects = random.choices(pnumbers,k=10)
  for i_num in range(len(suspects)):
    ph_number = re.sub(r"x\S+", "", suspects[i_num])
    ph_number = re.sub("[^0-9]", "", ph_number)[-10:]
    suspects[i_num] = ph_number

  # сохранение начального круга подозреваемых, '№ suspects' - начало нового круга
  with open("suspects.txt", "w+") as file:
    file.writelines('0 suspects\n')
    file.writelines([f'{str(number)}\n' for number in suspects])


  for call in calls:
    ph_number = re.sub(r"x\S+", "", call['caller'])
    ph_number = re.sub("[^0-9]", "", ph_number)[-10:]
    call['caller'] = ph_number
    ph_number = re.sub(r"x\S+", "", call['recipient'])
    ph_number = re.sub("[^0-9]", "", ph_number)[-10:]
    call['recipient'] = ph_number

  for i_num in range(len(suspects)):
    ph_number = re.sub(r"x\S+", "", suspects[i_num])
    ph_number = re.sub("[^0-9]", "", ph_number)[-10:]
    suspects[i_num] = ph_number


  # сохранение всех уникальных номеров в отфоматированном виде
  pnum = [re.sub("[^0-9]", "", re.sub(r"x\S+", "", number))[-10:] for number in pnumbers]
  with open("numbers.txt", "w+") as file:
    file.writelines([f'{str(number)}\n' for number in pnum])

  return calls

In [37]:
def find_suspects(calls, n=1, number_calls3 = False):
  """
  Нахождение нового круга подозреваемых
  calls: данные звонков вида: caller, recipient, duration_s
  n: количество кругов
  number_calls3: количество звонков подозреваемых == 3, если True, иначе == 1
  """

  # чтение последнего круга подозревамых
  suspects = []
  suspects_circles = {}
  circle = 0
  with open("suspects.txt") as file:
    for line in file:
      line_format = line.rstrip('\n')
      if line_format.find('suspects') > 0:
        suspects_circles[circle] = []
        circle += 1
      else:
        suspects_circles[circle-1].append(line_format)

  suspects = suspects_circles[circle-1]

  # отбор нового круга подозреваемых
  for i in range(n):
    new_suspects = [ {"caller": call["caller"], "recipient":  call["recipient"], "duration_s":  call["duration_s"]} for call in calls 
                                                            if call["recipient"] in suspects and call["duration_s"]> 120
                                                             ]
    # второе задание: три звонка подозреваемым
    if number_calls3:
      call_number = {}
      for call in new_suspects:
        if call['caller'] not in call_number.keys():
          call_number[call['caller']] = 0
        else:
          call_number[call['caller']] += 1
      new_suspects3 = set([call for call, num in call_number.items() if num > 2])
      # запись доказательств
      with open("proves.txt", "a") as file:
        file.writelines(f'{circle+i} suspects\n')
        file.writelines([f'caller:{call["caller"]}|recipient:{call["recipient"]}|duration_s:{call["duration_s"]}\n' for call in new_suspects
                                                                                                              if call['caller'] in new_suspects3])
      new_suspects = new_suspects3
    # первое задание: один звонок подозреваемым
    else:
      with open("proves.txt", "a") as file:
        file.writelines(f'{circle+i} suspects\n')
        file.writelines([f'caller:{call["caller"]}|recipient:{call["recipient"]}|duration_s:{call["duration_s"]}\n' for call in new_suspects])
      new_suspects = set([call['caller'] for call in new_suspects])

    # запись нового круга подозреваемых
    with open("suspects.txt", "a") as file:
      file.writelines(f'{circle+i} suspects\n')
      file.writelines([f'{str(number)}\n' for number in new_suspects])

    suspects = list(new_suspects)
    print(f'suspects {i}: {suspects}')


In [38]:
calls = generation_data(num_numbers = 200, num_calls = 100000)

In [39]:
calls

[{'caller': '3900724479', 'duration_s': 396, 'recipient': '6635752514'},
 {'caller': '7037656440', 'duration_s': 156, 'recipient': '1442280567'},
 {'caller': '3374195563', 'duration_s': 194, 'recipient': '7180263536'},
 {'caller': '0676687174', 'duration_s': 401, 'recipient': '4198602959'},
 {'caller': '0805726856', 'duration_s': 512, 'recipient': '9960285525'},
 {'caller': '9343480438', 'duration_s': 55, 'recipient': '9443022229'},
 {'caller': '6598053547', 'duration_s': 288, 'recipient': '4423168900'},
 {'caller': '4648923739', 'duration_s': 473, 'recipient': '3093919101'},
 {'caller': '6402461036', 'duration_s': 401, 'recipient': '8453165243'},
 {'caller': '8275624585', 'duration_s': 491, 'recipient': '0576322185'},
 {'caller': '5078766670', 'duration_s': 345, 'recipient': '1240137669'},
 {'caller': '0938401250', 'duration_s': 205, 'recipient': '7150772005'},
 {'caller': '5447300555', 'duration_s': 196, 'recipient': '3934166281'},
 {'caller': '6505234207', 'duration_s': 333, 'recipi

In [40]:
n = int(input('Enter number of circles\n'))
find_suspects(calls, n, True)

Enter number of circles3
suspects 0: ['1341536550', '6505234207', '4708491494', '0781813795', '7280294353', '8706162154', '6469505489', '6635752514', '7180263536', '7192315346', '4597890550', '5702253990', '6466104510', '9397647575', '3756343265', '3900724479', '7717398200', '6010531387', '4229180736', '9937588696', '7150772005', '9239396211', '2417763688', '4198602959', '7703718923', '4691237623', '7885544984', '9880121242', '8889771688', '4648923739', '4783110859', '4398163942', '9343480438', '0319904945', '8858571351', '6678355662', '3930193521', '9932081561', '2413001448', '7322965892', '5795382398', '0518475409', '3358094011', '0035398259', '5160085573', '4507902774', '9773572516', '4726068095', '5357527499', '8171980088', '6455723822', '2264824811', '6431057926', '5364532122', '0953618549', '0639364728', '3054255285', '4034790068', '0158232420', '6690903966', '8035853032', '9784245061', '6628165970', '8453165243', '0007127775', '2538019887', '7037656440', '4916513800', '338190472